## Group 8 Assignment Phase 2

In [1]:
//Start a simple Spark Session
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._

//Optional: Use the following code below to set the Error reporting
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)


//For Cleaning
//import scala.util.matching.Regex

val spark = SparkSession.builder().getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://22ef46be2451:4041
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1590564308401)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.log4j._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@d477063


## Load Monthly Fuel Price and Flight Delay Data

In [2]:
val flightDelaySchema = StructType(List(
    StructField("Route", StringType, nullable = false),
    StructField("Departing_Port", StringType, nullable = false),
    StructField("Arriving_Port", StringType, nullable = false),
    StructField("Airline", StringType, nullable = false),
    StructField("Month", StringType, nullable = false),
    StructField("Sectors_Scheduled", IntegerType, nullable = false),
    StructField("Sectors_Flown", IntegerType, nullable = false),
    StructField("Cancellations", IntegerType, nullable = false),
    StructField("Departures_On_Time", IntegerType, nullable = false),
    StructField("Arrivals_On_Time", IntegerType, nullable = false),
    StructField("Departures_Delayed", IntegerType, nullable = false),
    StructField("Arrivals_Delayed", IntegerType, nullable = false),
    StructField("Year", IntegerType, nullable = false),
    StructField("Month_Num", IntegerType, nullable = false)
))


val fuelDF = spark.read
    .option("delimiter",",")
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("jetfuel.csv")

val readflightDelayDF = spark
    .read
    .schema(flightDelaySchema)
    .option("delimiter",",")
    .option("header", "true")
    .csv("otp_time_series_web.csv")
    .where(($"Airline" =!= "All Airlines") )   //Remove All Ports from monthly totals
    .where($"Route" =!= "All Ports-All Ports") //Remove All Airlines from monthly totals
    .select("Month",
          "Departing_Port", 
          "Arriving_Port", 
          "Airline", 
          "Sectors_Flown",
          "Departures_Delayed",
          "Year", 
          "Month_Num")

flightDelaySchema: org.apache.spark.sql.types.StructType = StructType(StructField(Route,StringType,false), StructField(Departing_Port,StringType,false), StructField(Arriving_Port,StringType,false), StructField(Airline,StringType,false), StructField(Month,StringType,false), StructField(Sectors_Scheduled,IntegerType,false), StructField(Sectors_Flown,IntegerType,false), StructField(Cancellations,IntegerType,false), StructField(Departures_On_Time,IntegerType,false), StructField(Arrivals_On_Time,IntegerType,false), StructField(Departures_Delayed,IntegerType,false), StructField(Arrivals_Delayed,IntegerType,false), StructField(Year,IntegerType,false), StructField(Month_Num,IntegerType,false))
fuelDF: org.apache.spark.sql.DataFrame = [Month: string, Price: double ... 1 more field]
readflightDel...

In [3]:
readflightDelayDF
    .show(5)
    
readflightDelayDF
    .select("Departing_Port")
    .distinct()
    .show()

readflightDelayDF
    .select("Arriving_Port")
    .distinct()
    .show()

readflightDelayDF
    .select("Airline")
    .distinct()
    .show(truncate = false)

readflightDelayDF
    .select("Departures_Delayed")
    .describe()
    .show()

+------+--------------+-------------+-------+-------------+------------------+----+---------+
| Month|Departing_Port|Arriving_Port|Airline|Sectors_Flown|Departures_Delayed|Year|Month_Num|
+------+--------------+-------------+-------+-------------+------------------+----+---------+
|Jan-04|      Adelaide|     Brisbane| Qantas|           93|                12|2004|        1|
|Jan-04|      Adelaide|     Canberra| Qantas|           48|                 3|2004|        1|
|Jan-04|      Adelaide|   Gold Coast| Qantas|            9|                 1|2004|        1|
|Jan-04|      Adelaide|    Melbourne| Qantas|          350|                33|2004|        1|
|Jan-04|      Adelaide|        Perth| Qantas|          130|                 9|2004|        1|
+------+--------------+-------------+-------+-------------+------------------+----+---------+
only showing top 5 rows

+--------------+
|Departing_Port|
+--------------+
|  Port Lincoln|
|        Cairns|
|      Armidale|
|      Canberra|
|Sunshine 

## Join The Monthly Fuel Price

In [4]:
val flightDelayDF = readflightDelayDF.join(fuelDF, readflightDelayDF("Month") === fuelDF("Month"), "left")
flightDelayDF.count()

flightDelayDF: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
res1: Long = 61367


## Replicate Aggregated Late Departures with 1's and Not Lates with 0's

In [5]:
val df2 = flightDelayDF.withColumn("Departures_Delayed",
                            explode(array_repeat(lit(1), $"Departures_Delayed")))
//df2.show()

val df3 = flightDelayDF.withColumn("Departures_Delayed",
                            explode(array_repeat(lit(0), $"Sectors_Flown"-$"Departures_Delayed")))
//df3.show()

// Take a random sample (without replacement) of the data (to reduce memory requirements)
val sampleFraction = 0.3

//Concatenate rows df2 and df3 and drop any rows with missing data
val flights = (df2.union(df3)
                            .drop("Sectors_Flown", "Month", "Change")
                            .withColumnRenamed("Departures_Delayed","label")
                            .withColumnRenamed("Price","Fuel_Price")
                            .sample(false,sampleFraction).na.drop())

println(s"flights has ${flights.count()} rows")
flights.printSchema()

flights has 1887845 rows
root
 |-- Departing_Port: string (nullable = true)
 |-- Arriving_Port: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- label: integer (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month_Num: integer (nullable = true)
 |-- Fuel_Price: double (nullable = true)



df2: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
df3: org.apache.spark.sql.DataFrame = [Month: string, Departing_Port: string ... 9 more fields]
sampleFraction: Double = 0.3
flights: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 5 more fields]


## Determine the fraction of lates in the dataset

In [6]:
// Take a look at the proportion of lates in the dataset
val counts = flights.groupBy("label").count()

println("proportion of lates (label=1)")
counts.show()

proportion of lates (label=1)
+-----+-------+
|label|  count|
+-----+-------+
|    1| 320804|
|    0|1567041|
+-----+-------+



counts: org.apache.spark.sql.DataFrame = [label: int, count: bigint]


## Write the Dataframe flights to a file and Read it Back in

In [7]:
//flightDelay.parquet

//flights.write.parquet("flight_data.parquet")
flights.coalesce(1) //Join all partitions into one file
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("flight_data")

val flights1 = spark.read
  .option("delimiter",",")
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("flight_data")

flights1.printSchema()

flights1.show(5)

root
 |-- Departing_Port: string (nullable = true)
 |-- Arriving_Port: string (nullable = true)
 |-- Airline: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month_Num: integer (nullable = true)
 |-- Fuel_Price: double (nullable = true)

+--------------+-------------+-------+-----+----+---------+----------+
|Departing_Port|Arriving_Port|Airline|label|Year|Month_Num|Fuel_Price|
+--------------+-------------+-------+-----+----+---------+----------+
|      Adelaide|     Brisbane| Qantas|    1|2004|        1|       1.3|
|      Adelaide|     Brisbane| Qantas|    1|2004|        1|       1.3|
|      Adelaide|     Brisbane| Qantas|    1|2004|        1|       1.3|
|      Adelaide|     Brisbane| Qantas|    1|2004|        1|       1.3|
|      Adelaide|     Canberra| Qantas|    1|2004|        1|       1.3|
+--------------+-------------+-------+-----+----+---------+----------+
only showing top 5 rows



flights1: org.apache.spark.sql.DataFrame = [Departing_Port: string, Arriving_Port: string ... 5 more fields]
